### Setup 

In [ ]:
import numpy as np 
import tensorflow as tf 
from tensorflow import keras 
from keras import layers 

### Dowload and prepare datasets

In [ ]:
vocab_size = 20000# xem xét 20000 từ đầu tiên /  Only consider the top 20k words 
num_tokens_per_example = 200  # chỉ xem xét 200 từ đầu tiên của mỗi đánh giá
# Only consider the first 200 words of each movie review 
(x_train , y_train) , (x_val , y_val) = keras.datasets.imdb.load_data(num_words=vocab_size)
print(len(x_val) , 'Training sequences')
print(len(x_val), "Validation sequences")
# biến đổi đầu vào thành một chuỗi tuần tự
x_train = keras.utils.pad_sequences(
    x_train , maxlen=num_tokens_per_example
)

x_val = keras.utils.pad_sequences(x_val , maxlen=num_tokens_per_example)

### Define hyperparameters

In [ ]:
embed_dim = 32 # embedding size for each token
num_heads = 2  # number of attention heads 
ff_dim = 32 # hidden layers size in feedforward network 
num_experts = 10 # number expert used in the Switch Trasformer
batch_size = 50 
learning_rate = 0.001
dropout_rate = 0.25
num_epochs = 3
num_tokens_per_batch = (
    batch_size * num_tokens_per_example
) # total number of tokes per patch  / tổng số tokens của mỗi patch 
print(f'Number of tokens per patch : {num_tokens_per_batch}') # 10000

### Implement token & position embedding layer

In [ ]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen , vocab_size, embed_dim):
        super().__init__()
        # ở vector nhúng embeding nhận đầu vào là kích thước tập từ vựng
        # trả về số chiều nhúng 
        self.token_emb = layers.Embedding(
            input_dim=vocab_size , output_dim=embed_dim
        )
        # vector nhúng vị trí nhận vào kích thước độ dài mỗi vector = maxlen 
        # trả về số chiều nhúng 
        self.pos_emb = layers.Embedding(
            input_dim = maxlen ,output_dim = embed_dim
        )
    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0 , limit=maxlen, delta=0)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions 
        

### Implement the Feedforward network 

In [ ]:
def create_feedforward_network(ff_dim , name=None):
    return keras.Sequential(
        [
            layers.Dense(ff_dim , activation='relu'), 
            layers.Dense(ff_dim),
        ], 
        name = name ,
    )

### Implement the load_balanced loss

In [ ]:
def load_balanced_loss(router_probs , expert_mask):
    # loss = alpha.N . sum(1-> N). f(i) .p(i)
    # f(i) là tỷ lệ mã thông báo được gửi đến chuyên gia i 
    # p(i)là xác xuất bộ định tuyến được phân bổ cho chuyên gia i
    # hàm mất mát cân bằng tải nhận đầu vào là
    # router_probs [token_per_batch , num_expert] là xác xuất được chỉ định 
    # cho mỗi chuyên gia trên mã thông báo
    # expert_mask [tokens_per_batch, num_experts] chứa chuyên gia có xác xuất 
    # bộ định tuyến cao nhát dạng one hot 
    num_experts = tf.shape(expert_mask)[-1]
    # nhận phần nhỏ mã thông báo được gửi đến mỗi chuyên gia 
    # xác xuất là 1 vector với độ dài = num chuyên giá có tổng = 1
    # (fi)fi =1/T .sum (x E B) 1{argmax p(x) = i}
    density = tf.reduce_mean(expert_mask , axis=0)
    # nhận một phần khối lượng xác xuất được chỉ định cho từng chuyên gia
    # từ bộ định tuyến trên tất cả mã thông báo  xác xuất là 1 vector 
    # với độ dài = num chuyên giá có tổng = 1
    #(pi)
    density_probs = tf.reduce_mean(router_probs , axis=0)
    # muốn cả 2 vectors có phân bổ thống nhất 1 / num_expertsm trên tất cả 
    # num_expert phần tử . Hai vector sẽ được đẩy về phía phân bổ thống nhất 
    # khi tích vô hướng được thu nhỏ
    loss = tf.reduce_mean(density * density_probs) * tf.cast(
            (num_experts **2), tf.dtypes.float32,
    )
    return loss 
    

### Implement the router as a layer

In [ ]:
# tính toán phân bổ cho các chuyên gia và năng lực của chuyên gia 
class Router(layers.Layer):
    def __init__(self, num_experts , expert_capacity):
        self.num_experts = num_experts 
        self.route = layers.Dense(units=num_experts)
        self.expert_capacity = expert_capacity
        super().__init__()
    def call(self, inputs , training=False):
        # input shape = [tokens_per_patch , embed_dim]
        # router_logits shape = [tokens_per_patch , num_experst]
        # khởi tạo router logits = route(inputs)
        router_logits = self.route(inputs)
        if training:
            # Thêm nhiễu để khám phá giữa các chuyên gia 
            # một ma trận bằng chính nó với min = 0.9 max = 1.1
            router_logits += tf.random.uniform(
                shape=router_logits.shape , minval=0.9 , maxval=1.1
            )
        # tính xác xuất của mỗi mã thông báo mà nó được gửi tới các chuyên gia 
        # là xác xuất cao nhất trên chuyên gia qua softmax
        router_probs = keras.activations.softmax(router_logits, axis=-1)
        # sử dụng hàm top k để lấy ra expert có xác xuất cao nhất cho mỗi token 
        # Kết quả trả về 2 ma trận shape [token_per_patch, 1] , 
        # expert_gate chứa xác xuất cao nhất cho mã thông báo thứ i 
        # expert index là chỉ số của expert tương ứng 
        expert_gate , expert_index = tf.math.top_k(router_probs, k=1)
        # khởi tạo ma trận expert_mask shape [token_per_patch, num_experts]
        # trong expert[i, j] bằng 1 nếu token thứ i được gửi đến chuyên gia j 
        # và bằng 0 nếu ngược lại 
        expert_mask = tf.one_hot(expert_index , depth=self.num_experts)
        # tính toán hàm mất mát cân bằng tải 
        aux_loss= load_balanced_loss(router_probs, expert_mask)
        # Tính toán vị trí chuyên gia shape [ token_per_patch , num_experts]
        # position_un_expert là vị trí token thứ i trong hàng đợi j 
        # được tính bằng cách cộng dồn tổng giá trị theo chiều ngang 
        position_in_expert = tf.cast(
            tf.math.cumsum(expert_mask, axis=0) * expert_mask , tf.dtypes.int32
        )
        # giữ lại các mã thông báo phù hợp với năng lực của chuyên gia 
        # kiểm tra xem vị trí của mỗi token(position_in_expert) có nhỏ hơn expert_capacity không 
        # nếu có đoạn mã sẽ giữ nguyên giá trị trong expert_mask nếu ko giá trị 
        # trong expert_mask được gán  = 0
        # chỉ những token nhỏ hơn expert_mask mới được gửi đến expert , lớn hơn thì bỏ
        expert_mask *= tf.cast(
            tf.math.less(
                    tf.cast(position_in_expert , tf.dtypes.int32) , self.expert_capacity
            ),
            tf.dtypes.float32,
        )
        # tính toán expert_mask_flat là một ma trận có kích thước [token_per_patch]
        # trong đó phần tử expert_mask_flat[i] đại diện cho tổng số lượng các chuyên gia mà token thứ i được gửi đến 
        # vd tokens 1 sẽ được gửi đén  n chyên gia 
        # đoạn mã phục vụ để loại bỏ những token khônng được gửi đến expert nào 
        expert_mask_flat = tf.reduce_mean(expert_mask ,axis=-1)
        expert_gate *= expert_mask_flat 
        # tạo một tensor kết hợp shape [token_per_patch , num_expert , expert_capacity]
        # combined_tensor[i,j ,k] băng = 1 nếu token thứ i được gửi đến expert thứ j cps vị trí thứu k 
        # trong hàng đợi của expert mask đó 
        # sử dụng tf.sqeeze để loại bỏ các chiều có kích thước bằng 1 trong kích thước của 1 tensor 
        combined_tensor = tf.expand_dims(
            expert_gate * expert_mask_flat * 
            tf.squeeze(tf.one_hot(expert_index , depth=self.num_experts) , 1),
            -1
        )* tf.squeeze(tf.one_hot(position_in_expert, depth=self.expert_capacity),1)
        # xây dựng dispatch tensor là một ma trận nhị phân 
        # shape [token_per_patch , num_experts , expert_capacity] 
        #  = 1 nếu mã thông báo gửi đến chuyên gia tương ứng 
        dispatch_tensor = tf.cast(combined_tensor, tf.dtypes.float32)
        
        return dispatch_tensor, combined_tensor 
        

### Implement a Swicth layer 

In [ ]:
class Switch(layers.Layer):
    def __init__(self,num_experts, embed_dim , num_tokens_per_batch, capacity_factor=1):
        self.num_exprets = num_experts 
        self.num_tokens_per_batch = num_tokens_per_batch
        self.embed_dim = embed_dim 
        self.experts = [
            create_feedforward_network(embed_dim) for _ in range(num_experts)
        ]
        self.expert_capacity = num_tokens_per_batch // num_experts 
        self.route = Router(num_experts, self.expert_capacity)
        super().__init__()
        
    def call(self, inputs):
        # inputs shape ;[num_tokens_per_patch]
        batch_size = tf.shape(inputs)[0]
        num_tokens_per_xample = tf.shape(inputs)[1]
        
        # input shape : [num_tokens_per_patch , embed_dim]
        inputs = tf.reshape(inputs , [self.num_tokens_per_batch , self.embed_dim])
        # dispatch_tensor shape : [tokens_per_patch, num_experts, expert_capacity]
        # combine_tensor shape: [tokens_per_batch, num_experts, expert_capacity]
        dispatch_tensor, combined_tensor = self.route(inputs)
        # thiết lập đầu vào cho chuyên gia 
        # expert_input shape : [num_epxerts , expert_capacity , embed_dim]
        expert_inputs = tf.einsum("ab , acd->cdb" , inputs , dispatch_tensor)
        expert_inputs = tf.reshape(
            expert_inputs , [num_experts, self.expert_capacity, self.embed_dim]
        )
        # thiết lập danh sách tensor gửi đến chuyên gia 
        # gửi các token đến các chuyên gia bằng cách tách expert_input thành 1 danh
        # sách các tensor theo chiều ngang shape [expert_capacity , embed_dim]
        expert_input_list = tf.unstack(expert_inputs, axis=0)
        expert_output_list = [ 
            # chuyên gia tính toán đầu vào từ danh sách expert_input_list theo vị trí idx các mã thông báo 
            self.experts[idx](expert_input)
            for idx , expert_input in enumerate(expert_input_list)
        ]
        # thiết lập đầu ra của các chuyên gia 
        # nối các tensor theo danh sách ở trên theo chiều dọc 
        # shape mới  : [expert_capacity , num_expert , embed_dim]
        # chứa biểu diễn mới của tokens sau khi qua expert
        expert_outputs = tf.stack(expert_output_list , axis=1)
        # đầu ra kết hợp expert_output_combined shape :[token_per_patch ,embed_dim]
        expert_outputs_combined = tf.einsum(
                "abc,xba->xc", expert_outputs , combined_tensor # => [token_per_patch, embed_dim]
        )
        # output _shape [batch_size , num_token_per_example , embed_dim]
        # reshape output 
        outputs = tf.reshape (
            expert_outputs_combined , [batch_size , num_tokens_per_example , self.embed_dim]
        )
        
        return outputs 
        

### Implement a Transfoemer bolck layer

In [ ]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim , num_heads , ffn ,dropout_rate=0.1):
        super().__init__()
        self.embed_dim = embed_dim 
        self.num_heads = num_heads
        self.ffn = ffn
        self.attn = layers.MultiHeadAttention(
            num_heads = num_heads , key_dim = embed_dim
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(dropout_rate)
        self.dropout2 = layers.Dropout(dropout_rate)
        
    def call(self, inputs, training):
        attn_outputs = self.attn(inputs, inputs)
        att_outputs = self.dropout1(attn_outputs, training=training)
        out1 = self.layernorm1(inputs + attn_outputs)
        ffn_outputs = self.ffn(out1)
        ffn_outputs = self.dropout2(ffn_outputs, training=training)
        return self.layernorm2(out1 + ffn_outputs)

### Implement the classifier

In [ ]:
def create_classifier():
    switch = Switch(num_experts , embed_dim , num_tokens_per_batch)
    transformer_block = TransformerBlock(ff_dim , num_heads, switch)
    
    inputs = layers.Input((num_tokens_per_example ,))
    # thưucj hiện lớp nhúng từ và vị trí 
    embedding_layers = TokenAndPositionEmbedding(
        num_tokens_per_example , vocab_size , embed_dim
    )
    # xây dựng các lớp đầu đủ thành mô hình 
    x = embedding_layers(inputs)
    x = transformer_block(x)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dropout(dropout_rate)(x)
    x = layers.Dense(ff_dim , activation='relu')(X)
    x = layers.Dropout(dropout_rate)(x)
    x = layers.Dense(2 , activation='softmax')(x)
    
    outputs = x
    classifier = keras.Model(inputs=inputs, outputs = outputs)
    return classifier 

### Train and evaluate the model 

In [ ]:
def run_experiment(classifier):
    classifier.compile(
        optimizer= keras.optimizers.Adam(learning_rate),
        loss = "sparse_categorical_crossentropy",
        metrics = ['accuracy'],
    )
    history = classifier.fit(
        x_train , y_train,
        batch_size = batch_size,
        epochs = 3 , 
        validation_data = (x_val ,y_val)
    )
    return history 

classifier = create_classifier()
run_experiment(classifier)